In [1]:
from utils_nlp import Tools
tools = Tools('mhk9c')

installing package 1
installing package 2
installing package 3
installing package 4
installing package 5
Done Installing packages


In [2]:
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline
import sparknlp
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.clustering import LDA

In [3]:
import sparknlp
spark = sparknlp.start()

In [ ]:
# spark = SparkSession.builder \
#     .appName("Spark NLP")\
#     .config("spark.driver.memory","8G")\
#     .config("spark.driver.maxResultSize", "2G") \
#     .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.11:2.4.5")\
#     .config("spark.kryoserializer.buffer.max", "1000M")\
#     .getOrCreate()

In [5]:
_df = tools.load_data(spark, "russian-troll-tweets-enriched")
df = _df.filter(_df['label']==1)
df.printSchema()

Done loading from /project/ds5559/team1_sp22/data//russian-troll-tweets-enriched.
root
 |-- external_author_id: string (nullable = true)
 |-- author: string (nullable = true)
 |-- content: string (nullable = true)
 |-- region: string (nullable = true)
 |-- language: string (nullable = true)
 |-- publish_date: string (nullable = true)
 |-- harvested_date: string (nullable = true)
 |-- following: integer (nullable = true)
 |-- followers: integer (nullable = true)
 |-- updates: integer (nullable = true)
 |-- post_type: string (nullable = true)
 |-- account_type: string (nullable = true)
 |-- retweet: integer (nullable = true)
 |-- account_category: string (nullable = true)
 |-- new_june_2018: integer (nullable = true)
 |-- alt_external_id: string (nullable = true)
 |-- tweet_id: string (nullable = true)
 |-- article_url: string (nullable = true)
 |-- tco1_step1: string (nullable = true)
 |-- tco2_step1: string (nullable = true)
 |-- tco3_step1: string (nullable = true)
 |-- curated_conten

In [6]:
df.count()

1120229

In [7]:
# Spark NLP requires the input dataframe or column to be converted to document. 
document_assembler = DocumentAssembler() \
    .setInputCol("content") \
    .setOutputCol("document") \
    .setCleanupMode("shrink")

In [8]:
# Split sentence to tokens(array)
tokenizer = Tokenizer() \
  .setInputCols(["document"]) \
  .setOutputCol("token")

In [9]:
# clean unwanted characters and garbage
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")

In [10]:
# remove stopwords
stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)

In [11]:
# stem the words to bring them to the root form.
stemmer = Stemmer() \
    .setInputCols(["cleanTokens"]) \
    .setOutputCol("stem")

In [12]:
finisher = Finisher() \
    .setInputCols(["stem"]) \
    .setOutputCols(["tokens"]) \
    .setOutputAsArray(True) \
    .setCleanAnnotations(False)

In [13]:
# We build a ml pipeline so that each phase can be executed in sequence. This pipeline can also be used to test the model. 
nlp_pipeline = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            stemmer, 
            finisher])

In [14]:
# train the pipeline
nlp_model = nlp_pipeline.fit(df)

In [15]:
# apply the pipeline to transform dataframe.
processed_df  = nlp_model.transform(df)

In [ ]:
processed_df.printSchema()

In [ ]:
processed_df.count()

In [16]:
# tokens_df = processed_df.select('publish_date','tokens').limit(10000)
tokens_df = processed_df.select('publish_date','tokens')
tokens_df.show()

+---------------+--------------------+
|   publish_date|              tokens|
+---------------+--------------------+
|11/11/2015 2:41|[jebbush, dont, l...|
|11/11/2015 2:42|[your, talk, immi...|
|11/11/2015 4:22|[gopstop, gopdeb,...|
|11/11/2015 4:53|[adammathisss, sl...|
|11/11/2015 5:08|[teaparti, gopsto...|
|11/11/2015 5:09|[nowthisnew, gopd...|
|11/11/2015 5:14|[republican, raci...|
|11/11/2015 5:26|[imposs, watch, f...|
|12/7/2015 18:58|[excit, new, on, ...|
|12/7/2015 18:59|[reason, anyon, f...|
|12/7/2015 18:59|[watch, on, amaz,...|
|12/7/2015 18:59|[todai, join, u, ...|
|12/7/2015 18:59|[tomorrow, night,...|
|12/7/2015 18:59|[amaz, what, amaz...|
|12/7/2015 18:59|[mi, amiga, evalo...|
|12/7/2015 18:59|[ooher, imaceleb,...|
|12/7/2015 19:00|[true, httpstcowh...|
| 9/19/2015 8:23|[firearm, crimin,...|
| 9/19/2015 8:24|[second, amend, c...|
| 9/19/2015 8:24|[everyon, want, b...|
+---------------+--------------------+
only showing top 20 rows



In [17]:
cv = CountVectorizer(inputCol="tokens", outputCol="features", vocabSize=500, minDF=3.0)

In [18]:
# train the model
cv_model = cv.fit(tokens_df)

In [19]:
# transform the data. Output column name will be features.
vectorized_tokens = cv_model.transform(tokens_df)

In [20]:
num_topics = 10
lda = LDA(k=num_topics, maxIter=10)
model = lda.fit(vectorized_tokens)
ll = model.logLikelihood(vectorized_tokens)
lp = model.logPerplexity(vectorized_tokens)
print("The lower bound on the log likelihood of the entire corpus: " + str(ll))
print("The upper bound on perplexity: " + str(lp))

The lower bound on the log likelihood of the entire corpus: -27599930.324379705
The upper bound on perplexity: 6.290721487034518


In [21]:
# extract vocabulary from CountVectorizer
vocab = cv_model.vocabulary
topics = model.describeTopics()   
topics_rdd = topics.rdd
topics_words = topics_rdd\
       .map(lambda row: row['termIndices'])\
       .map(lambda idx_list: [vocab[idx] for idx in idx_list])\
       .collect()
for idx, topic in enumerate(topics_words):
    print("topic: {}".format(idx))
    print("*"*25)
    for word in topic:
       print(word)
    print("*"*25)

topic: 0
*************************
polic
new
kill
state
man
muslim
cop
offic
music
playe
*************************
topic: 1
*************************
get
hillari
time
u
dont
know
clinton
rt
let
year
*************************
topic: 2
*************************
rt
peopl
media
presid
realdonaldtrump
first
trump
on
love
us
*************************
topic: 3
*************************
black
via
trump
work
world
war
anoth
chang
protest
peopl
*************************
topic: 4
*************************
obama
trump
think
right
america
american
great
that
patriot
back
*************************
topic: 5
*************************
rt
cnn
break
maga
trump
new
watch
tcot
fakenew
potu
*************************
topic: 6
*************************
video
white
new
im
hous
rt
citi
pleas
top
amp
*************************
topic: 7
*************************
want
got
see
blacklivesmatt
real
well
right
rt
go
never
*************************
topic: 8
*************************
trump
rt
amp
call
ne
support
good
don